In [1]:
import pickle
import numpy as np 
import pandas as pd
from operator import itemgetter 
from sklearn.metrics import precision_recall_fscore_support

# فراخوانی مدل های مورد نیاز 

In [2]:
z0_Pred_clf=pickle.load(open('Pred_clf0.pkl','rb'))
z0_allModels_clf =pickle.load(open('allModels_clf0.pkl','rb'))
z0_Fscoer_clf=pickle.load(open('Fscoer_clf0.pkl','rb'))
z0_Pred_clf=pickle.load(open('Pred_clf0.pkl','rb'))

Ydev=pickle.load(open('Ydev.pkl','rb'))
Ytest=pickle.load(open('Ytest.pkl','rb'))


In [3]:
#best clasifire
all_M_D_frame =pd.DataFrame(z0_allModels_clf,columns=['Modle','Fscore'])
Best_Fscore=(max(z0_Fscoer_clf))

Best_Modle=all_M_D_frame[all_M_D_frame['Fscore']==Best_Fscore]

best_pre=z0_Pred_clf[Best_Modle.index[0]]

print('Best_Singleclf_Fscore=',Best_Fscore)

# voting

def modle_voting (Ydev,predict_list):
    Columns=[]
    for i in range(len(predict_list[0])) :
        Columns.append('pred%d'%i)
    df=pd.DataFrame(predict_list,columns=Columns)
    pre_final=[]
    for i in range (df.shape[1]):
        a=df['pred%d'%i].tolist()
        max_freq=np.bincount(a).tolist() 
        pre_final.append(max_freq.index(max(max_freq)))
    F_eclf1=precision_recall_fscore_support(Ydev,pre_final, average='micro')[2]
    return F_eclf1,pre_final 

best_score_com = modle_voting(Ydev,z0_Pred_clf)[0]    
best_pre_com = modle_voting(Ydev,z0_Pred_clf)[1]  

print('best_score_voting=',best_score_com )
# best_pre_com 


#forward
clf_forward=[]
for i in range (len(z0_allModels_clf)):
    temp=[]
    temp.append(z0_allModels_clf[i][1])
    temp.append(z0_allModels_clf[i][0])
    temp.append(z0_Pred_clf[i])
    clf_forward.append(temp)

sorted(clf_forward, key = itemgetter(0),reverse=False)
best_score_forward=clf_forward[0][0]
#print(best_score_forward)
best_pre_forward=[z0_Pred_clf[0]]

for i in range(len(z0_Pred_clf)):
    templist=best_pre_forward+[clf_forward[i][2]]
    p_voting= modle_voting (Ydev,templist)
    score_forward = p_voting[0]
    pre_forward =p_voting[1]
    
    if score_forward > best_score_forward:
        best_score_forward = score_forward
        best_pre_forward=templist  
    #print(best_score_forward)


print('best_fscore_Forward=',best_score_forward)
# best_pre_forward


#backward
z0_Pred_clf =pickle.load(open('Pred_clf0.pkl','rb'))
z0_allModels_clf =pickle.load(open('allModels_clf0.pkl','rb'))
clf_backward=[]
for i in range (len(z0_allModels_clf)):
    temp=[]
    temp.append(z0_allModels_clf[i][1])
    temp.append(z0_allModels_clf[i][0])
    temp.append(z0_Pred_clf[i])
    clf_backward.append(temp)
    

sorted(clf_backward, key = itemgetter(0),reverse=True)
predictbl=[]

for i in range(len(z0_Pred_clf)):
    predictbl.append(clf_backward[i][2])
    
backward= modle_voting(Ydev,z0_Pred_clf)    

best_score_back=backward[0] 
best_pre_back = [backward[1]] 

templist=list (range (len(predictbl)))
for i in range (1,len(predictbl)):
     templist[i]=predictbl[:-i]

for i in range(1,len(predictbl)):#
    p_voting= modle_voting (Ydev,templist[i])
    if best_score_back < p_voting[0]:
        best_score_back = p_voting[0]
        #print(best_score_back)
        best_pre_back=templist[i]

print('fscore_Backward=',best_score_back)
# best_pre_back


# comm




Best_Singleclf_Fscore= 0.810520939734423
best_score_voting= 0.8202247191011236
best_fscore_Forward= 0.822267620020429
fscore_Backward= 0.823799795709908


In [4]:
z0_Pred_clf=pickle.load(open('Pred_clf0.pkl','rb'))

def modle_voting (Ydev,predict_list):
    Columns=[]
    for i in range(len(predict_list[0])) :
        Columns.append('pred%d'%i)
    df=pd.DataFrame(predict_list,columns=Columns)
    pre_final=[]
    for i in range (df.shape[1]):
        a=df['pred%d'%i].tolist()
        max_freq=np.bincount(a).tolist() 
        pre_final.append(max_freq.index(max(max_freq)))
    F_eclf1=precision_recall_fscore_support(Ydev,pre_final, average='micro')[2]
    return F_eclf1,pre_final 


In [5]:
best_predict_final = best_pre_back+best_pre_forward+[np.array(best_pre_com)]+[np.array(best_pre.tolist(),dtype=float)]
best_score_final = modle_voting(Ydev,best_predict_final)[0] 
print('best_score_final=',best_score_final)

best_score_final= 0.8197139938712973
